In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationSummaryBufferMemory

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=180,
    return_messages=True,
)

examples = [
    {
        "movie_title": "Top Gun",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "movie_title": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝 ",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{movie_title}"), ("ai", "{answer}")]
)

fewshot_chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You always reply with three emojis"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{movie_title}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(movie_title):
    result = chain.invoke({"movie_title": movie_title})
    memory.save_context(
        {"input": movie_title},
        {"output": result.content},
    )
    print(result.content)

In [12]:
invoke_chain("Inception")

🌀🎬🧠


In [13]:
invoke_chain("Parasites")

🦠🔬💉


In [14]:
invoke_chain("Godzilla")

🦖🏙️🔥


In [15]:
invoke_chain("The Holiday")

🎄✈️❤️


In [16]:
invoke_chain(
    "What is all of the movies I asked before? please answer me with the emojis"
)

🌀🎬🧠
🦠🔬💉
🦖🏙️🔥
🎄✈️❤️


In [17]:
invoke_chain(
    "What is all of the movies I asked before? please answer me with the english title"
)

Inception
Parasite
Godzilla
The Holiday
